# Example 7: Sun-Synchronous Circular Orbit — Multi-Model Propagation


## Problem Statement
This example designs and simulates a Sun-synchronous orbit (SSO) at an altitude of 500 km.
The goal is to determine the required inclination for Sun-synchronicity, then propagate the orbit
using three models:
- Two-Body (ideal Keplerian motion)
- J2 Perturbation (Earth’s oblateness)
- HPOP (High-Fidelity Orbit Propagator with higher-order forces)

We then compare the trajectories and ground tracks to analyze the differences between the models.

## Initial Conditions
- Semi-major axis: $a = 6878.137$ km (Earth radius + 500 km altitude)
- Eccentricity: $e = 0$ (circular)
- Inclination: $i \approx 97.4^\circ$
- RAAN = $0^\circ$, Argument of Perigee = $0^\circ$, True Anomaly = $0^\circ$


## Governing Equations
Sun’s mean motion:
$$ \Omega_{req} = \frac{2\pi}{365.26 \times 24 \times 3600} \quad [rad/s] $$

J2-induced nodal regression:
$$ \dot{\Omega}_{J2} = -\tfrac{3}{2} J_2 \left(\tfrac{R_E}{a}\right)^2 \sqrt{\tfrac{\mu}{a^3}} \cos i $$

Sun-synchronicity requires:
$$ \cos i = -\tfrac{2}{3 J_2} \left(\tfrac{a}{R_E}\right)^2 \Omega_{req} \sqrt{\tfrac{a^3}{\mu}} $$

With altitude $h = 500$ km ($a = R_E + h = 6878.137$ km), this yields $i \approx 97.4^\circ$.

## Method
1. Compute the required inclination for a 500 km altitude Sun-synchronous orbit.
2. Convert Keplerian orbital elements to Cartesian state vectors.
3. Propagate the orbit over 5 days with a 120-second time step using:
   - Two-Body model
   - J2 Perturbation model
   - HPOP (High-Precision Orbit Propagator)
4. Generate 3D visualizations and combined ground-track plots to compare the models.



### Code

Here is the example code:

In [ ]:
import numpy as np
from datetime import datetime, timezone
from orbit_util import Orbit_2body, OrbitVisualizer

# Constants (fallbacks taken if not exposed by Orbit_2body)
orb = Orbit_2body()
mu  = getattr(orb, 'mu', 398600.4418)   # km^3/s^2
R_E = getattr(orb, 'R_E', 6378.137)     # km
J2  = getattr(orb, 'J2', 1.08263e-3)    # -

# Sun-synchronous design for h = 500 km
h_alt_km = 500.0
a = R_E + h_alt_km
Omega_dot_req = 2*np.pi / (365.26*24*3600.0)
cos_i = -(2.0/(3.0*J2))*((a/R_E)**2)*Omega_dot_req*np.sqrt(a**3/mu)
cos_i = np.clip(cos_i, -1.0, 1.0)
i_deg = float(np.degrees(np.arccos(cos_i)))

print(f"Designed SSO inclination ≈ {i_deg:.2f} deg (target ~97.4 deg)")
print(f"a = {a:.3f} km, mu = {mu:.3f} km^3/s^2, J2 = {J2}")


In [ ]:
# Build initial state from Keplerian elements
e = 0.0
RAAN = 0.0
w = 0.0
theta = 0.0
h_spec = np.sqrt(mu * a)
R0, V0 = orb.keplerian_to_cartesian(e=e, h=h_spec, theta=theta, i=i_deg, RAAN=RAAN, w=w, degree_mode=True)
print("Initial Cartesian state constructed.")


In [ ]:
# Propagation settings
T_days = 5           # Use 5 days here; increase if you want longer tracks
T = T_days * 24 * 3600
time_step = 120      # seconds
scenario_epoch = datetime.now(timezone.utc)
print(f"Propagating for {T_days} days with dt = {time_step} s ...")

# Two-Body
sol_2body, t_2body = orb.propagate_init_cond(T=T, time_step=time_step, R0=R0, V0=V0)

# J2
sol_j2, t_j2 = orb.propagate_with_J2(T=T, time_step=time_step, R0=R0, V0=V0)

# HPOP (ensure required kernels/params are configured in orbit_util)
sol_hpop, t_hpop = orb.HFOP(T=T, time_step=time_step, R0=R0, V0=V0, scenario_epoch=scenario_epoch)
print("Propagation finished.")


In [ ]:
# Extract ECI positions and compute ground tracks
r_2body = sol_2body[:, 0:3]
r_j2    = sol_j2[:, 0:3]
r_hpop  = sol_hpop[:, 0:3]

lat_2body, lon_2body = orb.lat_long_from_ECI(r_eci=r_2body, t=t_2body, scenario_epoch=scenario_epoch)
lat_j2,    lon_j2    = orb.lat_long_from_ECI(r_eci=r_j2,    t=t_j2,    scenario_epoch=scenario_epoch)
lat_hpop,  lon_hpop  = orb.lat_long_from_ECI(r_eci=r_hpop,  t=t_hpop,  scenario_epoch=scenario_epoch)
print("Latitude/longitude tracks computed.")


In [ ]:
# 3D visualizations
viz = OrbitVisualizer()
viz.EarthStatic(r=r_2body, title="Example 7 — Two‑Body Orbit (3D)", names=["Two‑Body"])
viz.EarthStatic(r=r_j2,    title="Example 7 — J2 Orbit (3D)",      names=["J2"])
viz.EarthStatic(r=r_hpop,  title="Example 7 — HPOP Orbit (3D)",    names=["HPOP"])
print("3D plots created.")


In [ ]:
# Combined ground‑track visualization
latitudes  = np.vstack([np.nan_to_num(lat_2body), np.nan_to_num(lat_j2), np.nan_to_num(lat_hpop)])
longitudes = np.vstack([np.nan_to_num(lon_2body), np.nan_to_num(lon_j2), np.nan_to_num(lon_hpop)])
viz.ground_track(latitudes=latitudes, longitudes=longitudes, names=["Two‑Body", "J2", "HPOP"])
print("Ground tracks plotted.")
